# Multiple Linear Regression
If our columns are numeric, it's straightforward to add them to a linear regression model. Below we are adding an extra feature to a model `GrLivArea`

In [ ]:
import pandas as pd
from utils import evaluate_model
from sklearn import linear_model

def train_multi_linear_model(training_set):
    def encode_data(data):
        return(data[['BedroomAbvGr','GrLivArea', 'LotArea']])
    predictor = linear_model.LinearRegression()
    predictor.fit(encode_data(training_set),training_set['SalePrice'])
    def multi_linear_model(input_data):
        return(predictor.predict(encode_data(input_data)))
    return(multi_linear_model)

In [ ]:
training_set = pd.read_csv("housing_price_data/training_data.csv")
multi_linear_model = train_multi_linear_model(training_set)
evaluate_model(multi_linear_model)

## Exercise

Modify the code above to add `LotArea` to the multilinear model and observe the performance of the model

## Working with categorical variables (Part 1)
In order to make use of categorical data, we first need to encode it as a number. As a simple example, we'll encode the variable `CentralAir` as 0 if there is no central air conditioning, and 1 if there is central air conditioning. We can do this by using a boolean comparison operation, and relying on the fact that `True == 1`

In [ ]:
def train_central_air_model(training_set):
    def encode_data(data):
        features = data.copy()
        features['CentralAir'] = features['CentralAir'] == 'Y'
        features = features[['BedroomAbvGr','GrLivArea','CentralAir']]
        return(features)

    training_features = encode_data(training_set)
    predictor = linear_model.LinearRegression()
    predictor.fit(training_features, training_set['SalePrice'])
    
    def central_air_model(input_data):
        input_features = encode_data(input_data)
        return(predictor.predict(input_features))
    return(central_air_model)

In [ ]:
central_air_model = train_central_air_model(training_set)
evaluate_model(central_air_model)

# Exercise

Add whether or not the house has a pool to this model in the code above. (Look at the `PoolArea` variable). Compare this model's accruacy now that the presence of a pool is a feature. Did it improve it or make it worse? 

# Working with ordinal variables
If the categories of a variable follow a clear rank, then we can label them by this rank. An example of this is the basement quality column.

    BsmtCond: Evaluates the general condition of the basement

       Ex	Excellent
       Gd	Good
       TA	Typical - slight dampness allowed
       Fa	Fair - dampness or some cracking or settling
       Po	Poor - Severe cracking, settling, or wetness

We would encode this as Po:1, Fa:2, TA:3, Gd:4, and Ex:5.

For houses without a basement (i.e. `BsmtCond is NaN`), we use a default value of 0


In [ ]:
def train_basement_cond_model(training_set):
    def encode_data(data):
        features = data.copy()
        bsmt_cond_map = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1}
        features['BsmtQuality'] = features['BsmtCond'].map(bsmt_cond_map).fillna(0) # Some houses have no basement
        
        features = features[['BedroomAbvGr','GrLivArea','BsmtQuality']]
        return(features)

    training_features = encode_data(training_set)
    predictor = linear_model.LinearRegression()
    predictor.fit(training_features, training_set['SalePrice'])
    
    def basement_cond_model(input_data):
        input_features = encode_data(input_data)
        return(predictor.predict(input_features))
    return(basement_cond_model)

In [ ]:
basement_cond_model = train_basement_cond_model(training_set)
evaluate_model(basement_cond_model)

## Exercise

Add to the code above the KitchenQuality as a feature (see `KitchenQual` in `data_description.txt`)

# Working with categorical variables (part 2)


If there is no order, we use a technique called 'one hot encoding'. This involves creating a new column for each category

For example, the `Electrical` variable in the housing prices dataset contains the following categories:

       SBrkr	Standard Circuit Breakers & Romex
       FuseA	Fuse Box over 60 AMP and all Romex wiring (Average)	
       FuseF	60 AMP Fuse Box and mostly Romex wiring (Fair)
       FuseP	60 AMP Fuse Box and mostly knob & tube wiring (poor)
       Mix	Mixed

We would encode the table

|Electrical|
|---------|
  |FuseA|
  |FuseF|
  |FuseP|
  |Mix|
  |SBrkr|
  
as

|FuseA |FuseF|FuseP|Mix|SBrkr|
|-----|-----|-----|-----|
| 1| 0  | 0   | 0  | 0  |
| 0|1  | 0   | 0  | 0  |
| 0|0  | 1  | 0| 0|
| 0|0  | 0 | 1 |0 |
| 0|0  | 0 | 0 | 1|

In [ ]:
def encode_electrical(electrical):
    one_hot_encoding = pd.DataFrame()
    one_hot_encoding['FuseA'] = electrical == 'FuseA'
    one_hot_encoding['FuseF'] = electrical == 'FuseF'
    one_hot_encoding['FuseP'] = electrical == 'FuseP'
    one_hot_encoding['Mix']   = electrical == 'Mix'
    one_hot_encoding['SBrkr'] = electrical == 'SBrkr'
    return(one_hot_encoding)


In [ ]:
encode_electrical(training_set['Electrical']).head(20)

In [ ]:
def train_electrical_model(training_set):
    def encode_data(data):
        electrical = encode_electrical(data['Electrical'])
        data = pd.concat([electrical,data[["BedroomAbvGr","GrLivArea"]]],axis=1)
        return(data)
    training_features = encode_data(training_set)
    predictor = linear_model.LinearRegression()
    predictor.fit(training_features, training_set['SalePrice'])
    
    def electrical_model(input_data):
        input_features = encode_data(input_data)
        return(predictor.predict(input_features))
    return(electrical_model)

electrical_model = train_electrical_model(training_set)
evaluate_model(electrical_model)


## Exericse

Encode `Heating` using One Hot encoding

# Scaling and combining features

Many machine learning algorithms require all variables to be on the same scale, ideally between -1 and 1. Let's compare LotArea to SalePrice

In [ ]:
training_set[['LotArea','SalePrice']].describe()

We can transform these variables to be on the same scale using a preprocessing trick called _min/max scaling_. 

`MinMaxScale(X) = (X - min(X))/(max(X) - min(X))` 

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
scaled = pd.DataFrame(scaler.fit_transform(training_set[['LotArea','SalePrice']]),
             columns=['LotArea','SalePrice'])
scaled.head()

In [ ]:
scaled.describe()

We can put these back to their original scale using the `inverse_transform` method.

In [ ]:
pd.DataFrame(scaler.inverse_transform(scaled),
             columns=["LotArea","SalePrice"]).head()

In [ ]:
training_set[["LotArea","SalePrice"]].head()

We can also add new features to our model by combining two or more existing features. For example, let's
multiply bedrooms by bathrooms. 
We'll also scale our input and output.

In [ ]:
def train_bedroom_bathroom_model(training_set):
    def encode_data(data,scaler = None):
        data = data.copy()
        data = data[['FullBath','BedroomAbvGr']]
        data['BedBath'] = data['FullBath'] * data['BedroomAbvGr']
        
        if(not scaler):
            scaler = MinMaxScaler()
            scaler.fit(data)
        pd.DataFrame(scaler.transform(data), columns = ['FullBath','BedroomAbvGr','BedBath'])
        return(data,scaler)
    
    training_features, scaler = encode_data(training_set)
    predictor = linear_model.LinearRegression()
    predictor.fit(training_features, training_set['SalePrice'])
    
    def bedroom_bathroom_model(input_data):
        input_features,_ = encode_data(input_data,scaler)
        predictions = predictor.predict(input_features)
        return(predictions)
    return(bedroom_bathroom_model)

In [ ]:
bedroom_bathroom_model = train_bedroom_bathroom_model(training_set)
evaluate_model(bedroom_bathroom_model)

# Competition Exercise

Build a linear model to achieve as low score as possible. Winner will receive an Eliiza water bottle.